### 2. Вірусні новини

У статті [Automatic Extraction of News Values from Headline Text](http://www.aclweb.org/anthology/E17-4007) описано основні ознаки заголовків, які кидаються в очі і змушують читача таки прочитати новину:
1. іменовані сутності (імена людей, назви компаній тощо)
2. емоційне забарвлення
3. вищий і найвищий ступені порівняння
4. близькість до читача
5. елемент несподіванки
6. унікальність

**Завдання:**
1. Напишіть програму, яка аналізує заголовок за першими трьома ознаками (у спрощеній формі)
   * Чи є в заголовку іменовані стуності?
   * Чи є заголовок позитивно чи негативно забарвлений?
   * Чи є в заголовку прикметники та прислівники вищого і найвищого ступенів порівняння?
2. Проженіть вашу програму на [корпусі заголовків з The Examiner](data/examiner-headlines.txt). Для кожної з трьох ознак визначте відсоток заголовків у корпусі, які її мають.
3. Збережіть програму та пораховану статистику в директорії з вашим іменем.

Додаткова інформація:
- Типи сутностей, які впливають на "вірусність" заголовка, виберіть самостійно.
- Для визначення емоційного забарвлення, використайте [SentiWordNet](https://github.com/aesuli/sentiwordnet). Наприклад, можна перевірити, що середнє значення позитивності/негативності слова у заголовку перевищує 0.5. Для визначення середнього значення можна брати до п'яти перших значень слова з такою частиною мови. Будьте креативними та експериментуйте. Можна користуватися SentiWordNet з бібліотеки [NLTK](http://www.nltk.org/howto/sentiwordnet.html).

In [1]:
import spacy
import en_core_web_sm

from nltk.corpus import sentiwordnet as swn

nlp = spacy.load('en_core_web_sm')

In [2]:
VENTS = ['PERSON', 'ORG', 'NORP', 'GPE', 'EVENT', 'MONEY', 'WORK_OF_ART']

def has_viral_ents(doc):
    return any(e.label_ in VENTS for e in doc.ents)

assert has_viral_ents(nlp('Barack Obama was born in Hawaii.')) == True
assert has_viral_ents(nlp('Very boring uknown fact without any name.')) == False

In [3]:
WNET_POS = {'NOUN': 'n', 'VERB': 'v', 'ADJ' : 'a', 'ADV' : 'r'}

def sentiment_score(t):
    # IDEA: take top 5 similar words
    ssets = list(swn.senti_synsets(t.lemma_, WNET_POS[t.pos_]))[:5]
    if len(ssets) == 0: return 0
    return sum(max(s.pos_score(), s.neg_score()) for s in ssets) / len(ssets)

def has_sentiment(doc):
    return any(sentiment_score(t) > 0.5 for t in doc if t.pos_ in WNET_POS) 

assert has_sentiment(nlp('The best film you will ever see.')) == True
assert has_sentiment(nlp("Hofstadter's Law: It always takes longer than you expect, even when you take into account Hofstadter's Law.")) == False

In [4]:
STAGS = ['JJR', 'JJS', 'RBR', 'RBS']

def is_suprelative(t):
    return t.tag_ in STAGS

def has_suprelative(doc):
    return any(is_suprelative(t) for t in doc)

assert has_suprelative(nlp("The best film you will ever see.")) == True
assert has_suprelative(nlp("Nothing special here.")) == False

In [5]:
exam_set = '../../../tasks/02-structural-linguistics/data/examiner-headlines.txt'

from collections import defaultdict

with open(exam_set) as headlines:
    total = 0
    result = defaultdict(int)
    
    for h in headlines:
        total += 1
        doc = nlp(h)
        if has_viral_ents(doc): result['nent'] += 1
        if has_sentiment(doc): result['sentim'] += 1
        if has_suprelative(doc): result['suprel'] += 1
            
    for metric, count in result.items():
        print('{} - {:.1%}'.format(metric, count / total))

nent - 66.2%
suprel - 4.6%
sentim - 8.7%
